<a href="https://colab.research.google.com/github/shokuhatsugi/task-1-jp-morgan/blob/main/swtestingmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preprocessing**


In [1]:
import pandas as pd

In [2]:
path = "/content/drive/MyDrive/Version_2/Version_2/Testing_carlease.csv"
data= pd.read_csv(path)
data.head()

,B_Req,R_Priority,FP,Complexity,Time,Cost,Prioirty,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,1,94,"TC#2027,TC#2928,TC#2053",3,4.0,60.0,Low,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,197,"TC#3269,TC#1752,TC#1042",3,4.0,60.0,Medium,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,163,"TC#2843,TC#3332,TC#805,TC#2785",3,5.0,75.0,Low,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,103,"TC#1618,TC#953,TC#1368",1,4.0,20.0,High,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,70,"TC#235,TC#3170,TC#2423",5,4.0,100.0,Medium,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
!pip install keras

In [8]:
!pip install -q keras

In [9]:
import keras

In [10]:
import tensorflow

In [12]:
from tensorflow import keras

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
user_stories = data['B_Req'].astype(str).values
test_cases = data['FP'].astype(str).values

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(np.concatenate((user_stories, test_cases), axis=0))

In [19]:

user_story_seqs = tokenizer.texts_to_sequences(user_stories)
test_case_seqs = tokenizer.texts_to_sequences(test_cases)

In [20]:
max_user_story_len = max(len(seq) for seq in user_story_seqs)
max_test_case_len = max(len(seq) for seq in test_case_seqs)

In [21]:
user_story_seqs = pad_sequences(user_story_seqs, maxlen=max_user_story_len, padding='post')
test_case_seqs = pad_sequences(test_case_seqs, maxlen=max_test_case_len, padding='post')

In [22]:
vocab_size = len(tokenizer.word_index) + 1

In [23]:
decoder_target_data = np.zeros_like(test_case_seqs)
decoder_target_data[:, :-1] = test_case_seqs[:, 1:]

**Model Definition**

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Attention

In [25]:
embedding_dim = 256
lstm_units = 512

In [26]:
encoder_inputs = Input(shape=(max_user_story_len,))
encoder_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

In [27]:
attention = Attention()
attention_result = attention([encoder_outputs, encoder_outputs])

In [28]:
decoder_inputs = Input(shape=(max_test_case_len,))
decoder_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])

In [29]:
decoder_concat_input = tf.concat([attention_result, decoder_outputs], axis=-1)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

ValueError: Exception encountered when calling layer "tf.concat" (type TFOpLambda).

Dimension 1 in both shapes must be equal, but are 1 and 8. Shapes are [?,1] and [?,8]. for '{{node tf.concat/concat}} = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32](Placeholder, Placeholder_1, tf.concat/concat/axis)' with input shapes: [?,1,512], [?,8,512], [] and with computed input tensors: input[2] = <-1>.

Call arguments received by layer "tf.concat" (type TFOpLambda):
  • values=['tf.Tensor(shape=(None, 1, 512), dtype=float32)', 'tf.Tensor(shape=(None, 8, 512), dtype=float32)']
  • axis=-1
  • name=concat

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, AdditiveAttention, Concatenate

In [31]:
embedding_dim = 256
lstm_units = 512

In [32]:
encoder_inputs = Input(shape=(max_user_story_len,))
encoder_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

In [33]:
decoder_inputs = Input(shape=(max_test_case_len,))
decoder_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])

In [34]:
attention = AdditiveAttention()
attention_result = attention([decoder_outputs, encoder_outputs])
decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attention_result])

In [35]:
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

In [36]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [37]:
model.fit([user_story_seqs, test_case_seqs], decoder_target_data, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
25/25 [==============================] - 31s 742ms/step - loss: 6.8524 - val_loss: 5.7852
Epoch 2/100
25/25 [==============================] - 15s 592ms/step - loss: 4.9264 - val_loss: 4.9885
Epoch 3/100
25/25 [==============================] - 15s 601ms/step - loss: 4.2719 - val_loss: 4.7973
Epoch 4/100
25/25 [==============================] - 15s 589ms/step - loss: 4.1162 - val_loss: 4.9102
Epoch 5/100
25/25 [==============================] - 15s 586ms/step - loss: 3.9874 - val_loss: 5.1008
Epoch 6/100
25/25 [==============================] - 15s 613ms/step - loss: 3.8461 - val_loss: 5.3693
Epoch 7/100
25/25 [==============================] - 16s 627ms/step - loss: 3.7333 - val_loss: 5.5900
Epoch 8/100
25/25 [==============================] - 15s 591ms/step - loss: 3.6008 - val_loss: 5.7821
Epoch 9/100
25/25 [==============================] - 15s 596ms/step - loss: 3.4574 - val_loss: 5.9538
Epoch 10/100
25/25 [==============================] - 15s 596ms/step - loss: 3.318

In [38]:
model.save('test_case_generator.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


**Evaluation and Inference**

In [44]:
def generate_test_case(user_story, tokenizer, model, max_user_story_len, max_test_case_len):
    user_story_seq = tokenizer.texts_to_sequences([user_story])
    user_story_seq = pad_sequences(user_story_seq, maxlen=max_user_story_len, padding='post')

    test_case_seq = np.zeros((1, max_test_case_len))
    test_case_seq[0, 0] = tokenizer.word_index.get('<start>', 1)  # assuming <start> token exists

    for i in range(1, max_test_case_len):
        output_tokens = model.predict([user_story_seq, test_case_seq])
        sampled_token_index = np.argmax(output_tokens[0, i-1, :])
        test_case_seq[0, i] = sampled_token_index

        if sampled_token_index == tokenizer.word_index.get('<end>', 2):  # assuming <end> token exists
            break

    test_case = tokenizer.sequences_to_texts(test_case_seq)
    return ' '.join(test_case)

In [47]:
new_user_story = "11"
test_case = generate_test_case(new_user_story, tokenizer, model, max_user_story_len, max_test_case_len)
print(test_case)

1/1 [==============================] - 0s 52ms/step
tc 118 tc 883 tc 2819


In [48]:
new_user_story = "10"
test_case = generate_test_case(new_user_story, tokenizer, model, max_user_story_len, max_test_case_len)
print(test_case)

1/1 [==============================] - 0s 54ms/step
tc 1783 tc 3377 tc 9
